In [33]:
import time
import datetime
from math import sqrt, exp, log
from random import gauss
import scipy.stats

In [34]:
##Definition of function
def generate_asset_price(S_t1,v,r,t1,t2):
    # S is spot price
    #V is implied vol
    T= (t2- t1).days/365.0
    return S_t1 * exp((r - 0.5 * v**2) * T + v * sqrt(T) * gauss(0,1.0))
def df(r,t1,t2):
    # function of discount factor between date t1 and t2 with interest rate r
    return exp(-r * (t2-t1).days/365.0)
    


In [43]:
S =100 # asset price at the start date
v= 0.20
r = 0.05
N = 1000
date = []
start_date = datetime.date(2012,6,4)
anniversary1 = datetime.date(2013,6,5)
anniversary2 = datetime.date(2014,6,7)
anniversary3 = datetime.date(2015,6,5)
anniversary4 = datetime.date(2016,6,7)
anniversary5 = datetime.date(2017,6,7)
end_date = datetime.date(2018,6,5)
date.append(start_date)
date.append(anniversary1)
date.append(anniversary2)
date.append(anniversary3)
date.append(anniversary4)
date.append(anniversary5)
date.append(end_date)
simulations = 1000000
asset_prices = [S] * 7 
l = len(asset_prices)
coupon_barrier = 0.8
kickout_barrier = 1.1
protection_barrier = 0.6
coupon_rate = 0.088
autocall_prices= []

In [46]:
for i in range(simulations):
    autocall_price = 0
    checker = True
    payoffs = [0] *l # on remet tous les payoffs a zero
    #Le payoff a la start_date vaut toujours zero
    
    for j in range(1, l- 1) :
        asset_prices[j]= generate_asset_price(asset_prices[j-1],v,r,date[j-1],date[j])/S
        if asset_prices[j] < coupon_barrier:
            payoffs[j+1] = payoffs[j]+ N * coupon_rate
            payoffs[j] = 0
            autocall_price += payoffs[j] * df(r,start_date,date[j])
        elif asset_prices[j] < kickout_barrier:
            payoffs[j] += N * coupon_rate
            autocall_price += payoffs[j]  * df(r,start_date,date[j])
        else:
            payoffs[j] += N * coupon_rate + N
            autocall_price += payoffs[j] * df(r,start_date,date[j])
            checker = False
            break
    
    if checker: 
        if asset_prices[l -1] < protection_barrier:
            payoffs[l - 1] = N * asset_prices[l -1]
        elif asset_prices[l -1] < coupon_barrier: 
            payoffs[l - 1] = N
        else:
            payoffs[l - 1] += N * coupon_rate + N
    
    autocall_prices.append(autocall_price)

final_autocall_price = sum(autocall_prices)/float(simulations)


    

In [49]:
print(final_autocall_price)

858.4082863995498
